In [18]:
import sys
sys.path.insert(1,'D:/Workspace/Project_VNNIC')
from transformers import AutoTokenizer , AutoModelForSequenceClassification
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.modeling_outputs import SequenceClassifierOutput
from main.processing_data.getLexicalFeature import * 

tokenizer = AutoTokenizer.from_pretrained('gechim/phobert-base-v2-finetuned')
phoBert = AutoModel.from_pretrained("gechim/phobert-base-v2-finetuned")

class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.phoBert = phoBert # (batchsize , 1 , 768)
    self.num_classes = num_classes
    self.fc1 = nn.Linear(input_size, 256)
    self.fc2 = nn.Linear(256, 768) #(batchsize , 1 , 768)
    self.dropout_nn = nn.Dropout(0.1)
    self.dropout_lm = nn.Dropout(0.1)


    # self.out = nn.Linear(768, num_classes)
    self.out = nn.Linear(1536, num_classes)

  def forward(self, features, input_ids, token_type_ids, attention_mask , labels):
    # output bên sang
    x_nn = F.relu(self.fc1(features))
    x_nn = F.relu(self.fc2(x_nn))

    # output bên bảo
    x_phoBert = self.phoBert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).last_hidden_state[:,0,:]

    #drop out trước khi concat
    x_nn = self.dropout_nn(x_nn)
    x_phoBert = self.dropout_lm(x_phoBert)

    # print(x_phoBert.shape)
    logits = self.out(torch.cat(( x_nn , x_phoBert) , dim=1)) #self.out( x_nn + x_phoBert)


    # tính loss cái này chỉ để hiện kq loss tập valid
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_classes), labels.view(-1))
    return SequenceClassifierOutput(loss = loss , logits=logits) # hàm trainer cần cái này nó mới chịu train

print("Model loaded \n\n\n")
model = torch.load('D:/Workspace/Project_VNNIC/models/model_concat_dataV2.pt' , map_location=torch.device('cpu'))
print(model)
print("\n\n\n")



def predict(url):
    print(getLexicalInputNN(url))
    url = normalize_url(url)
    url_tokenize = tokenizer(url , return_tensors='pt')
    x_feature = torch.tensor([getLexicalInputNN(url)] , dtype=torch.float32)
    y = model(features = x_feature,input_ids = url_tokenize['input_ids'] , token_type_ids = url_tokenize['token_type_ids'] , attention_mask = url_tokenize['attention_mask']  , labels = torch.tensor([1])).logits
    if torch.argmax(y).item() == 0:
        return "Bình thường"
    if torch.argmax(y).item() == 1:
        return "Có tín nhiệm thấp"
    
    

Some weights of RobertaModel were not initialized from the model checkpoint at gechim/phobert-base-v2-finetuned and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded 



NN(
  (phoBert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [21]:
print(predict("bet.com.vn"))

Error loading dictionary: [Errno 2] No such file or directory: 'datas\\data_train_nn\\dict_check_type.xlsx'
con_lai
[3.    1.585 0.    0.    0.    0.    0.    0.    0.    1.   ]
Error loading dictionary: [Errno 2] No such file or directory: 'datas\\data_train_nn\\dict_check_type.xlsx'
con_lai
Có tín nhiệm thấp
